In [ ]:

# ============================================================================
# IMPORTS
# ============================================================================
import sys
import os
import cv2
import numpy as np
import torch
import pyodbc
import pandas as pd
from PyQt5.QtWidgets import (
    QApplication, QWidget, QVBoxLayout, QMainWindow, QPushButton, QFileDialog,
    QTableWidget, QTableWidgetItem, QLabel, QGridLayout, QScrollArea, QMessageBox, 
    QDialog, QLineEdit, QHBoxLayout, QHeaderView
)
from PyQt5.QtGui import (
    QPixmap, QFont, QIcon, QImage, QPalette, QBrush
)
from PyQt5.QtCore import QObject, pyqtSignal, Qt

# ============================================================================
# CONSTANTS & CONFIGURATION
# ============================================================================
class Config:
    """Centralized configuration for the application"""
    # Window sizes
    WINDOW_MAIN = (1900, 900)
    WINDOW_LOGIN = (1400, 600)
    WINDOW_MODEL_OVERVIEW = (1500, 1200)
    WINDOW_STRUCTURE = (800, 700)
    WINDOW_IMAGE_DISPLAY = (800, 600)
    WINDOW_ACCESS_DB = (800, 600)
    WINDOW_REAL_TIME_DB = (800, 600)
    
    # Colors
    COLOR_BUTTON_NORMAL = '#B0BBC1'
    COLOR_BUTTON_HOVER = 'lightgray'
    COLOR_BACKGROUND = '#345C7F'
    COLOR_DARK_BG = '#104675'
    COLOR_GRAY_BUTTON = 'gray'
    
    # Font
    FONT_FAMILY = 'Times New Roman'
    FONT_SIZE = 14
    FONT_SIZE_LARGE = 25
    FONT_WEIGHT = QFont.Bold
    
    # Paths
    PATH_MODEL = 'The_last_2_of_Phantom_and_bayre/exp/weights/last.pt'
    PATH_DATABASE = r'Ac\Homam.accdb'
    PATH_IMAGES_P = 'images/P'
    PATH_IMAGES_BAYRAKTAR = 'images/Bayraktar'
    PATH_IMAGES_PHANTOM = 'images/Phantom'
    PATH_IMAGES_REAL_TIME = 'Our_DataBase'
    
    # Model image folders
    MODEL_IMAGE_PATHS = {
        'confusion_matrix': 'images/model/confusion_matrix',
        'F1_curve': 'images/model/F1_curve',
        'labels': 'images/model/labels',
        'labels_correlogram': 'images/model/labels_correlogram',
        'P_curve': 'images/model/P_curve',
        'PR_curve': 'images/model/PR_curve',
        'R_curve': 'images/model/R_curve',
        'results': 'images/model/results'
    }
    
    # Background images
    BG_MAIN = "images/EFGwdoSkHqhIgoyJiZIyXpHtPoq9l5KYkR2uYZu-JtQ.png"
    BG_MODEL_OVERVIEW = "images/is-machine-learning-hard-a-guide-to-getting-started-scaled-1-scaled.jpeg"
    BG_STRUCTURE = "images/LOP2.png"
    BG_LOGIN_IMAGE = "images/Drone-500x300-1-400x240.jpg"

# ============================================================================
# UTILITY FUNCTIONS
# ============================================================================
def create_styled_button(text, color_normal=None, color_hover=None, width=None, height=60):
    """Create a consistently styled button"""
    color_normal = color_normal or Config.COLOR_BUTTON_NORMAL
    color_hover = color_hover or Config.COLOR_BUTTON_HOVER
    
    button = QPushButton(text)
    button.setFont(QFont(Config.FONT_FAMILY, Config.FONT_SIZE, Config.FONT_WEIGHT))
    
    style = f"""
        QPushButton {{
            background-color: {color_normal};
            color: black;
            border: 2px solid gray;
            border-radius: 10px;
            margin-bottom: 0px;
            margin-top: 0px;
        }}
        QPushButton:hover {{
            background-color: {color_hover};
        }}
    """
    
    button.setStyleSheet(style)
    
    if width:
        button.setFixedWidth(width)
    if height:
        button.setFixedHeight(height)
    
    return button

def setup_window_background(window, image_path, width, height):
    """Set up background image for a window"""
    if os.path.exists(image_path):
        pixmap = QPixmap(image_path).scaled(width, height)
        palette = window.palette()
        palette.setBrush(QPalette.Background, QBrush(pixmap))
        window.setPalette(palette)
        window.setFixedSize(pixmap.size())
    else:
        print(f"Warning: Background image not found: {image_path}")

def create_image_grid(folder_path, parent_widget, image_size=(300, 300), columns=3):
    """Create a scrollable grid of images from a folder"""
    scroll_area = QScrollArea(parent_widget)
    scroll_area.setWidgetResizable(True)
    
    scroll_widget = QWidget()
    scroll_area.setWidget(scroll_widget)
    scroll_widget.setStyleSheet(f"background-color: {Config.COLOR_BACKGROUND};")
    
    grid_layout = QGridLayout(scroll_widget)
    grid_layout.setHorizontalSpacing(20)
    grid_layout.setVerticalSpacing(20)
    
    if os.path.exists(folder_path):
        images = [f for f in os.listdir(folder_path) 
                 if f.lower().endswith(('.jpg', '.png', '.jpeg'))]
        
        for index, image_name in enumerate(images):
            row = index // columns
            col = index % columns
            image_path = os.path.join(folder_path, image_name)
            
            try:
                pixmap = QPixmap(image_path)
                if not pixmap.isNull():
                    pixmap = pixmap.scaled(image_size[0], image_size[1], 
                                          Qt.KeepAspectRatio, Qt.SmoothTransformation)
                    image_label = QLabel()
                    image_label.setPixmap(pixmap)
                    image_label.setAlignment(Qt.AlignCenter)
                    grid_layout.addWidget(image_label, row, col)
            except Exception as e:
                print(f"Error loading image {image_name}: {e}")
    else:
        error_label = QLabel(f"Папка не найдена: {folder_path}")
        error_label.setStyleSheet("color: white; font-size: 16px;")
        grid_layout.addWidget(error_label, 0, 0)
    
    return scroll_area

# ============================================================================
# APPLICATION CLASSES
# ============================================================================
class OverviewOfModelForm(QWidget):
    """Form for displaying model overview and metrics"""
    def __init__(self):
        super().__init__()
        self.setWindowTitle('Обзор силы модели')
        self.initUI()

    def initUI(self):
        self.setGeometry(50, 50, *Config.WINDOW_MODEL_OVERVIEW)
        setup_window_background(self, Config.BG_MODEL_OVERVIEW, 1500, 800)
        
        layout = QVBoxLayout(self)
        
        # Create model metric buttons
        button_configs = [
            ('Матрица путаницы', 30, 220, 'confusion_matrix'),
            ('Кривая F1', 260, 150, 'F1_curve'),
            ('Метки', 420, 150, 'labels'),
            ('Коррелограмма \n меток', 585, 200, 'labels_correlogram'),
            ('Кривая P', 800, 150, 'P_curve'),
            ('Кривая PR', 960, 150, 'PR_curve'),
            ('Кривая R', 1120, 150, 'R_curve'),
            ('Результаты', 1280, 150, 'results')
        ]
        
        self.buttons = {}
        for text, x, width, key in button_configs:
            button = create_styled_button(text, width=width, height=60)
            button.setGeometry(x, 6, width, 60)
            button.clicked.connect(lambda checked, k=key: self.open_model_image(k))
            layout.addWidget(button)
            self.buttons[key] = button
        
        self.setLayout(layout)

    def open_model_image(self, image_type):
        """Open images for a specific model metric"""
        folder_path = Config.MODEL_IMAGE_PATHS.get(image_type)
        if folder_path and os.path.exists(folder_path):
            self.show_image_grid(folder_path)
        else:
            QMessageBox.warning(self, 'Ошибка', f'Папка не найдена: {folder_path}')

    def show_image_grid(self, folder_path):
        """Display images in a grid layout"""
        # Clear existing widgets except buttons
        for i in reversed(range(self.layout().count())):
            widget = self.layout().itemAt(i).widget()
            if widget not in self.buttons.values():
                widget.setParent(None)
        
        # Create and add scroll area with images
        scroll_area = create_image_grid(folder_path, self, image_size=(1200, 600), columns=3)
        scroll_area.setFixedSize(1400, 600)
        self.layout().addWidget(scroll_area)
        
        self.adjustSize()

class StructureOfObjectsForm(QWidget):
    """Form for displaying object structure images"""
    def __init__(self):
        super().__init__()
        self.setWindowTitle('Структура объектов')
        self.initUI()

    def initUI(self):
        self.setGeometry(100, 100, *Config.WINDOW_STRUCTURE)
        setup_window_background(self, Config.BG_STRUCTURE, 800, 700)
        
        layout = QVBoxLayout(self)
        
        # Bayraktar button
        self.bayraktar_button = create_styled_button(
            'Структура Bayraktar_TB2', 
            color_normal=Config.COLOR_GRAY_BUTTON,
            width=290,
            height=60
        )
        self.bayraktar_button.setGeometry(130, 6, 290, 60)
        self.bayraktar_button.clicked.connect(lambda: self.show_images(Config.PATH_IMAGES_BAYRAKTAR))
        layout.addWidget(self.bayraktar_button)
        
        # Phantom button
        self.phantom_button = create_styled_button(
            'Структура Phantom_4',
            color_normal=Config.COLOR_GRAY_BUTTON,
            width=245,
            height=60
        )
        self.phantom_button.setGeometry(450, 6, 245, 60)
        self.phantom_button.clicked.connect(lambda: self.show_images(Config.PATH_IMAGES_PHANTOM))
        layout.addWidget(self.phantom_button)
        
        self.setLayout(layout)

    def show_images(self, folder_path):
        """Display images from the specified folder"""
        # Clear existing widgets except buttons
        for i in reversed(range(self.layout().count())):
            widget = self.layout().itemAt(i).widget()
            if widget not in [self.bayraktar_button, self.phantom_button]:
                widget.setParent(None)
        
        # Create and add scroll area with images
        scroll_area = create_image_grid(folder_path, self, image_size=(600, 600), columns=3)
        scroll_area.setFixedSize(800, 500)
        self.layout().addWidget(scroll_area)
        
        self.adjustSize()

class ImageDisplayForm(QWidget):
    """Form for displaying analyzed images database"""
    def __init__(self):
        super().__init__()
        self.setWindowTitle('База данных проанализированных изображений')
        self.setGeometry(200, 200, *Config.WINDOW_IMAGE_DISPLAY)
        self.initUI()

    def initUI(self):
        layout = QVBoxLayout()
        
        # Create scroll area with images
        scroll_area = create_image_grid(Config.PATH_IMAGES_P, self, image_size=(300, 300), columns=3)
        layout.addWidget(scroll_area)
        
        self.setLayout(layout)

class ResultsWindow(QMainWindow):
    """Window for displaying analysis results"""
    def __init__(self, image_path=None, folder_path=None, video_path=None, 
                 model_path=None, image_size=(400, 400)):
        super().__init__()
        self.setWindowTitle('Результаты анализа')
        self.image_size = image_size
        
        # Load model if provided
        if model_path and os.path.exists(model_path):
            try:
                self.model = torch.hub.load('ultralytics/yolov5', 'custom', 
                                           path=model_path, force_reload=True)
            except Exception as e:
                QMessageBox.critical(self, 'Ошибка', f'Не удалось загрузить модель: {e}')
                self.model = None
        else:
            self.model = None
        
        # Create central widget and layout
        central_widget = QWidget()
        self.setCentralWidget(central_widget)
        layout = QVBoxLayout(central_widget)
        
        # Analyze based on input
        if image_path:
            self.analyze_single_image(image_path, layout)
        elif folder_path:
            self.analyze_images_in_folder(folder_path, layout)
        elif video_path:
            self.analyze_video(video_path, layout)
        else:
            QMessageBox.information(self, 'Информация', 'Нет данных для анализа')

    def analyze_single_image(self, image_path, layout):
        """Analyze a single image"""
        if os.path.exists(image_path) and self.model:
            try:
                results = self.model(image_path)
                self.display_results(results, layout)
            except Exception as e:
                QMessageBox.critical(self, 'Ошибка', f'Ошибка анализа: {e}')
        else:
            QMessageBox.warning(self, 'Предупреждение', 'Файл не найден или модель не загружена')

    def analyze_images_in_folder(self, folder_path, layout):
        """Analyze all images in a folder"""
        if os.path.exists(folder_path) and self.model:
            images = [f for f in os.listdir(folder_path) 
                     if f.lower().endswith(('.jpg', '.png', '.jpeg'))]
            
            for filename in images:
                image_path = os.path.join(folder_path, filename)
                try:
                    results = self.model(image_path)
                    self.display_results(results, layout)
                except Exception as e:
                    print(f"Error analyzing {filename}: {e}")
        else:
            QMessageBox.warning(self, 'Предупреждение', 'Папка не найдена или модель не загружена')

    def analyze_video(self, video_path, layout):
        """Analyze video (placeholder - would need video analyzer)"""
        QMessageBox.information(self, 'Информация', 'Анализ видео требует отдельного обработчика')

    def display_results(self, results, layout):
        """Display analysis results"""
        if results:
            # Display text results
            text_results = str(results)
            text_label = QLabel(text_results, self)
            text_label.setWordWrap(True)
            layout.addWidget(text_label)
            
            # Display image with detections
            try:
                img_array = np.squeeze(results.render())
                img_array_resized = cv2.resize(img_array, self.image_size)
                height, width, channel = img_array_resized.shape
                bytesPerLine = 3 * width
                qImg = QImage(img_array_resized.data, width, height, 
                             bytesPerLine, QImage.Format_RGB888)
                pixmap = QPixmap.fromImage(qImg)
                img_label = QLabel(self)
                img_label.setPixmap(pixmap)
                img_label.setAlignment(Qt.AlignCenter)
                layout.addWidget(img_label)
            except Exception as e:
                QMessageBox.warning(self, 'Предупреждение', f'Ошибка отображения изображения: {e}')

class VideoAnalyzer(QObject):
    """Video analysis worker"""
    video_analysis_finished = pyqtSignal()
    
    def __init__(self):
        super().__init__()
        self.model = None
        
    def load_model(self):
        """Load YOLOv5 model"""
        if os.path.exists(Config.PATH_MODEL):
            try:
                self.model = torch.hub.load('ultralytics/yolov5', 'custom', 
                                           path=Config.PATH_MODEL, force_reload=True)
                return True
            except Exception as e:
                print(f"Error loading model: {e}")
                return False
        return False
    
    def detect_and_save_objects(self, input_path, new_width=640, new_height=480):
        """Detect objects in video and save them"""
        if not self.load_model():
            print("Failed to load model")
            return
        
        if not os.path.exists(input_path):
            print(f"Input path not found: {input_path}")
            return
        
        # Create output directory if it doesn't exist
        os.makedirs(Config.PATH_IMAGES_REAL_TIME, exist_ok=True)
        
        cap = cv2.VideoCapture(input_path)
        frame_count = 0
        
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            
            # Resize frame
            frame = cv2.resize(frame, (new_width, new_height))
            
            # Make detections
            results = self.model(frame)
            detected_objects = results.xyxy[0]
            
            # Save detected objects
            for obj in detected_objects:
                x1, y1, x2, y2, confidence, class_id = obj
                object_image = frame[int(y1):int(y2), int(x1):int(x2)]
                
                # Only save if object is large enough
                if object_image.shape[0] > 10 and object_image.shape[1] > 10:
                    object_filename = os.path.join(
                        Config.PATH_IMAGES_REAL_TIME, 
                        f"object_{frame_count}_{int(x1)}_{int(y1)}.jpg"
                    )
                    cv2.imwrite(object_filename, object_image)
            
            frame_count += 1
            
            # Display frame (optional)
            cv2.imshow('Видеоанализ', np.squeeze(results.render()))
            
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        
        cap.release()
        cv2.destroyAllWindows()
        self.video_analysis_finished.emit()

class RealTimeDatabaseForm(QWidget):
    """Form for displaying real-time database images"""
    def __init__(self):
        super().__init__()
        self.setWindowTitle('База данных реального времени')
        self.setGeometry(200, 200, *Config.WINDOW_REAL_TIME_DB)
        self.initUI()

    def initUI(self):
        layout = QVBoxLayout()
        
        # Create scroll area with images
        scroll_area = create_image_grid(Config.PATH_IMAGES_REAL_TIME, self, 
                                       image_size=(300, 300), columns=3)
        layout.addWidget(scroll_area)
        
        self.setLayout(layout)

class AccessDatabaseForm(QMainWindow):
    """Form for displaying Access database data"""
    def __init__(self):
        super().__init__()
        self.setWindowTitle('База данных обнаружений')
        self.setGeometry(200, 200, *Config.WINDOW_ACCESS_DB)
        self.setStyleSheet(f"background-color: #f0f0f0;")
        self.initUI()

    def initUI(self):
        self.central_widget = QWidget()
        self.setCentralWidget(self.central_widget)
        
        layout = QVBoxLayout()
        
        # Create table widget
        self.table_widget = QTableWidget()
        self.table_widget.setStyleSheet(f"background-color: {Config.COLOR_BUTTON_NORMAL};")
        layout.addWidget(self.table_widget)
        
        # Load database data
        self.load_access_data()
        
        self.central_widget.setLayout(layout)

    def load_access_data(self):
        """Load data from Access database"""
        if os.path.exists(Config.PATH_DATABASE):
            try:
                conn_str = f'DRIVER={{Microsoft Access Driver (*.mdb, *.accdb)}};DBQ={Config.PATH_DATABASE};'
                conn = pyodbc.connect(conn_str)
                cursor = conn.cursor()
                
                cursor.execute("SELECT * FROM Detections")
                rows = cursor.fetchall()
                columns = [column[0] for column in cursor.description]
                
                # Display data in table
                self.table_widget.setRowCount(len(rows))
                self.table_widget.setColumnCount(len(columns))
                self.table_widget.setHorizontalHeaderLabels(columns)
                
                self.table_widget.horizontalHeader().setStyleSheet("background-color: #d9d9d9;")
                self.table_widget.horizontalHeader().setSectionResizeMode(QHeaderView.Stretch)
                
                for i, row in enumerate(rows):
                    for j, value in enumerate(row):
                        item = QTableWidgetItem(str(value) if value is not None else "")
                        self.table_widget.setItem(i, j, item)
                
                conn.close()
                
            except Exception as e:
                QMessageBox.critical(self, 'Ошибка', f'Ошибка загрузки базы данных: {e}')
        else:
            QMessageBox.warning(self, 'Предупреждение', f'Файл базы данных не найден: {Config.PATH_DATABASE}')

class LoginWindow(QDialog):
    """Login and registration window"""
    def __init__(self):
        super().__init__()
        self.setWindowTitle('Вход или регистрация')
        self.setGeometry(200, 200, *Config.WINDOW_LOGIN)
        self.setStyleSheet(f"background-color:{Config.COLOR_DARK_BG}; color: #f9f9f9;")
        self.initUI()

    def initUI(self):
        layout = QGridLayout()
        
        # Login section
        login_username_label = QLabel('Имя пользователя:')
        login_password_label = QLabel('Пароль:')
        self.login_username_edit = QLineEdit()
        self.login_password_edit = QLineEdit()
        self.login_password_edit.setEchoMode(QLineEdit.Password)
        login_button = QPushButton('Вход')
        
        # Style login widgets
        for label in [login_username_label, login_password_label]:
            label.setStyleSheet(f"font: {Config.FONT_SIZE_LARGE}px '{Config.FONT_FAMILY}'; color: #f9f9f9; margin-bottom: 5px;")
        
        edit_style = f"""
            font: bold {Config.FONT_SIZE_LARGE}px '{Config.FONT_FAMILY}';
            padding: 8px;
            border: 2px solid #ccc;
            border-radius: 10px;
            background-color: #444;
            color: #f9f9f9;
        """
        
        self.login_username_edit.setStyleSheet(edit_style)
        self.login_password_edit.setStyleSheet(edit_style)
        self.login_username_edit.setFixedWidth(230)
        self.login_password_edit.setFixedWidth(230)
        
        login_button.setStyleSheet(f"""
            QPushButton {{
                background-color: #858585;
                color: white;
                padding: 15px 30px;
                border: none;
                border-radius: 10px;
                font-family: '{Config.FONT_FAMILY}';
                font-weight: bold;
                font-size: {Config.FONT_SIZE_LARGE}px;
            }}
            QPushButton:hover {{
                background-color: #606060;
            }}
        """)
        
        # Add login widgets to layout
        layout.addWidget(login_username_label, 0, 0)
        layout.addWidget(self.login_username_edit, 0, 1)
        layout.addWidget(login_password_label, 1, 0)
        layout.addWidget(self.login_password_edit, 1, 1)
        layout.addWidget(login_button, 2, 1)
        
        # Registration section
        register_username_label = QLabel('Имя пользователя:')
        register_email_label = QLabel('Электронная почта:')
        register_phone_label = QLabel('Номер телефона:')
        register_password_label = QLabel('Пароль:')
        
        self.register_username_edit = QLineEdit()
        self.register_email_edit = QLineEdit()
        self.register_phone_edit = QLineEdit()
        self.register_password_edit = QLineEdit()
        self.register_password_edit.setEchoMode(QLineEdit.Password)
        register_button = QPushButton('Регистрация')
        
        # Style registration widgets
        for label in [register_username_label, register_email_label, 
                     register_phone_label, register_password_label]:
            label.setStyleSheet(f"font: {Config.FONT_SIZE_LARGE}px '{Config.FONT_FAMILY}'; color: #f9f9f9; margin-bottom: 5px;")
        
        for edit in [self.register_username_edit, self.register_email_edit,
                    self.register_phone_edit, self.register_password_edit]:
            edit.setStyleSheet(edit_style)
            edit.setFixedWidth(300)
        
        register_button.setStyleSheet(f"""
            QPushButton {{
                background-color: #858585;
                color: white;
                padding: 15px 30px;
                border: none;
                border-radius: 10px;
                font-family: '{Config.FONT_FAMILY}';
                font-weight: bold;
                font-size: {Config.FONT_SIZE_LARGE}px;
            }}
            QPushButton:hover {{
                background-color: #606060;
            }}
        """)
        
        # Add registration widgets to layout
        layout.addWidget(register_username_label, 0, 2)
        layout.addWidget(self.register_username_edit, 0, 3)
        layout.addWidget(register_email_label, 1, 2)
        layout.addWidget(self.register_email_edit, 1, 3)
        layout.addWidget(register_phone_label, 2, 2)
        layout.addWidget(self.register_phone_edit, 2, 3)
        layout.addWidget(register_password_label, 3, 2)
        layout.addWidget(self.register_password_edit, 3, 3)
        layout.addWidget(register_button, 4, 3)
        
        # Add image
        if os.path.exists(Config.BG_LOGIN_IMAGE):
            pixmap = QPixmap(Config.BG_LOGIN_IMAGE).scaled(600, 600)
            image_label = QLabel()
            image_label.setPixmap(pixmap)
            layout.addWidget(image_label, 0, 4, 5, 1)
        else:
            print(f"Login image not found: {Config.BG_LOGIN_IMAGE}")
        
        self.setLayout(layout)
        
        # Connect buttons
        login_button.clicked.connect(self.login)
        register_button.clicked.connect(self.register)

    def login(self):
        """Handle login"""
        username = self.login_username_edit.text().strip()
        password = self.login_password_edit.text().strip()
        
        if not username or not password:
            QMessageBox.warning(self, 'Вход не выполнен', 'Пожалуйста, введите как имя пользователя, так и пароль.')
            return
        
        if os.path.exists(Config.PATH_DATABASE):
            try:
                connection_string = f'DRIVER={{Microsoft Access Driver (*.mdb, *.accdb)}};DBQ={Config.PATH_DATABASE};'
                conn = pyodbc.connect(connection_string)
                cursor = conn.cursor()
                
                cursor.execute("SELECT Username, Password FROM Users WHERE Username=? AND Password=?", 
                              (username, password))
                user = cursor.fetchone()
                
                if user:
                    self.accept()
                else:
                    QMessageBox.warning(self, 'Вход не выполнен', 'Неправильное имя пользователя или пароль')
                
                cursor.close()
                conn.close()
                
            except Exception as e:
                QMessageBox.critical(self, 'Ошибка', f'Произошла ошибка: {str(e)}')
        else:
            QMessageBox.warning(self, 'Ошибка', f'База данных не найдена: {Config.PATH_DATABASE}')

    def register(self):
        """Handle registration"""
        username = self.register_username_edit.text().strip()
        password = self.register_password_edit.text().strip()
        email = self.register_email_edit.text().strip()
        phone = self.register_phone_edit.text().strip()
        
        if not all([username, password, email, phone]):
            QMessageBox.warning(self, 'Регистрация не удалась', 'Пожалуйста заполните все поля.')
            return
        
        if os.path.exists(Config.PATH_DATABASE):
            try:
                connection_string = f'DRIVER={{Microsoft Access Driver (*.mdb, *.accdb)}};DBQ={Config.PATH_DATABASE};'
                conn = pyodbc.connect(connection_string)
                cursor = conn.cursor()
                
                # Check if username exists
                cursor.execute("SELECT Username FROM Users WHERE Username=?", (username,))
                existing_user = cursor.fetchone()
                
                if existing_user:
                    QMessageBox.warning(self, 'Регистрация не удалась', 'Имя пользователя уже занято')
                else:
                    # Insert new user
                    cursor.execute("INSERT INTO Users (Username, Password, Email, Phone_Number) VALUES (?, ?, ?, ?)",
                                  (username, password, email, phone))
                    conn.commit()
                    
                    QMessageBox.information(self, 'Регистрация прошла успешно', 
                                           'Новый пользователь зарегистрирован успешно')
                    
                    # Clear fields
                    self.register_username_edit.clear()
                    self.register_password_edit.clear()
                    self.register_email_edit.clear()
                    self.register_phone_edit.clear()
                
                cursor.close()
                conn.close()
                
            except Exception as e:
                QMessageBox.critical(self, 'Ошибка', f'Произошла ошибка: {str(e)}')
        else:
            QMessageBox.warning(self, 'Ошибка', f'База данных не найдена: {Config.PATH_DATABASE}')

class MainWindow(QWidget):
    """Main application window"""
    def __init__(self):
        super().__init__()
        self.initUI()

    def initUI(self):
        self.setGeometry(100, 100, *Config.WINDOW_MAIN)
        self.setWindowTitle('Проект обнаружения дронов')
        
        # Set background
        setup_window_background(self, Config.BG_MAIN, *Config.WINDOW_MAIN)
        
        # Create buttons
        button_configs = [
            ('Анализ изображений', 50, 235, self.open_image_results_window),
            ('Видео анализ', 300, 200, self.open_video_results_window),
            ('База данных\nобнаружений', 515, 225, self.open_database_form),
            ('База данных\nреального времени', 750, 240, self.open_real_time_form),
            ('База данных\nпроанализированных изображений', 1000, 380, self.open_image_display_form),
            ('Структура объектов', 1390, 230, self.open_structure_form),
            ('Обзор силы модели', 1630, 250, self.open_model_form)
        ]
        
        self.buttons = {}
        for text, x, width, callback in button_configs:
            button = create_styled_button(text, width=width, height=70)
            button.setGeometry(x, 9, width, 70)
            button.clicked.connect(callback)
            self.buttons[text] = button
        
        self.show()

    def open_model_form(self):
        """Open model overview form"""
        self.model_form = OverviewOfModelForm()
        self.model_form.show()

    def open_image_results_window(self):
        """Open file dialog for image analysis"""
        options = QFileDialog.Options()
        file_dialog = QFileDialog()
        file_dialog.setFileMode(QFileDialog.ExistingFiles)
        file_dialog.setNameFilter("Images (*.jpg *.png)")
        
        filenames, _ = file_dialog.getOpenFileNames(
            self, "Выберите изображение или папку", "", 
            "Images (*.jpg *.png);;All Files (*)", options=options
        )
        
        if filenames:
            if os.path.exists(Config.PATH_MODEL):
                self.results_window = ResultsWindow(
                    image_path=filenames[0], 
                    model_path=Config.PATH_MODEL, 
                    image_size=(800, 600)
                )
                self.results_window.show()
            else:
                QMessageBox.warning(self, 'Ошибка', f'Модель не найдена: {Config.PATH_MODEL}')

    def open_video_results_window(self):
        """Open file dialog for video analysis"""
        options = QFileDialog.Options()
        file_dialog = QFileDialog()
        file_dialog.setFileMode(QFileDialog.ExistingFiles)
        file_dialog.setNameFilter("Videos (*.mp4 *.avi *.mov)")
        
        filenames, _ = file_dialog.getOpenFileNames(
            self, "Выберите видео", "", 
            "Videos (*.mp4 *.avi *.mov)", options=options
        )
        
        if filenames:
            self.video_analyzer = VideoAnalyzer()
            self.video_analyzer.video_analysis_finished.connect(self.open_real_time_form)
            
            # Run in a separate thread to avoid UI freezing
            import threading
            thread = threading.Thread(
                target=self.video_analyzer.detect_and_save_objects,
                args=(filenames[0],)
            )
            thread.daemon = True
            thread.start()

    def open_real_time_form(self):
        """Open real-time database form"""
        self.real_time_form = RealTimeDatabaseForm()
        self.real_time_form.show()

    def open_database_form(self):
        """Open Access database form"""
        self.database_form = AccessDatabaseForm()
        self.database_form.show()

    def open_image_display_form(self):
        """Open analyzed images form"""
        self.image_display_form = ImageDisplayForm()
        self.image_display_form.show()

    def open_structure_form(self):
        """Open object structure form"""
        self.structure_form = StructureOfObjectsForm()
        self.structure_form.show()

# ============================================================================
# MAIN EXECUTION
# ============================================================================
if __name__ == '__main__':
    app = QApplication(sys.argv)
    
    # Optional: Set application style
    app.setStyle('Fusion')
    
    # Create and show login window
    login_window = LoginWindow()
    
    # If login successful, show main window
    if login_window.exec_() == QDialog.Accepted:
        main_window = MainWindow()
        main_window.show()
        
        # Center window on screen
        screen = app.primaryScreen()
        screen_geometry = screen.geometry()
        window_geometry = main_window.frameGeometry()
        
        center_point = screen_geometry.center()
        window_geometry.moveCenter(center_point)
        main_window.move(window_geometry.topLeft())
        
        sys.exit(app.exec_())